# Классификация изображений

In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers.core import Dense

In [43]:
filenames = os.listdir("../data/cats_dogs")

In [44]:
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

In [45]:
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
df.head(10)

,filename,category
0,cat.0.jpg,0
1,cat.1.jpg,0
2,cat.10.jpg,0
3,cat.100.jpg,0
4,cat.1000.jpg,0
5,cat.10000.jpg,0
6,cat.10001.jpg,0
7,cat.10002.jpg,0
8,cat.10003.jpg,0
9,cat.10004.jpg,0


In [46]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'})

In [47]:
train_df, validate_df = train_test_split(df, test_size=0.20)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [48]:
validate_df = validate_df.sample(n=100).reset_index()
train_df = train_df.sample(n=1600).reset_index()
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=16
epochs=5

## VGG16

In [49]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [50]:
train_data_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "../data/cats_dogs/", 
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size
)

Found 1600 validated image filenames belonging to 2 classes.


In [51]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [52]:
test_data_generator = test_datagen.flow_from_dataframe(
    validate_df, 
    "../data/cats_dogs", 
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size
)

Found 100 validated image filenames belonging to 2 classes.


In [53]:
vgg16 = VGG16(weights='imagenet', include_top=True)
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [54]:
new_classification_layer = Dense(2, activation='softmax', name="CatsAndDogsPredictions")
out = new_classification_layer(vgg16.layers[-2].output)
model_from_vvg16 = Model(vgg16.input, out)

In [55]:
model_from_vvg16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [56]:
for l, layer in enumerate(model_from_vvg16.layers[:-1]):
    layer.trainable = False

model_from_vvg16.layers[-1].trainable = True

In [57]:
model_from_vvg16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [58]:
model_from_vvg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
model_from_vvg16.fit(train_data_generator, epochs=epochs, validation_data=test_data_generator,
                               validation_steps=total_validate//batch_size, steps_per_epoch=total_train//batch_size)

Epoch 1/5
100/100 [==============================] - 554s 6s/step - loss: 0.5659 - accuracy: 0.7181 - val_loss: 0.4216 - val_accuracy: 0.8229
Epoch 2/5
100/100 [==============================] - 537s 5s/step - loss: 0.4228 - accuracy: 0.8062 - val_loss: 0.4185 - val_accuracy: 0.8333
Epoch 3/5
100/100 [==============================] - 549s 5s/step - loss: 0.3804 - accuracy: 0.8269 - val_loss: 0.3641 - val_accuracy: 0.8229
Epoch 4/5
100/100 [==============================] - 532s 5s/step - loss: 0.3455 - accuracy: 0.8475 - val_loss: 0.3304 - val_accuracy: 0.8854
Epoch 5/5
100/100 [==============================] - 553s 6s/step - loss: 0.3295 - accuracy: 0.8612 - val_loss: 0.3193 - val_accuracy: 0.8958


In [60]:
loss, accuracy = model_from_vvg16.evaluate_generator(test_data_generator, total_validate//batch_size, workers=12)
print("accuracy = %f, loss = %f " % (accuracy, loss))

C:\Users\danya\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
accuracy = 0.895833, loss = 0.319603 


### Fine-tuning

In [61]:
for layer in model_from_vvg16.layers:
    if layer.name == 'block5_conv3':
        layer.trainable = True
    else:
        layer.trainable = False

model_from_vvg16.layers[-1].trainable = True

In [62]:
model_from_vvg16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [63]:
model_from_vvg16.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
model_from_vvg16.fit_generator(train_data_generator, epochs=epochs, validation_data=test_data_generator,
                               validation_steps=total_validate//batch_size, steps_per_epoch=total_train//batch_size)

C:\Users\danya\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/5
100/100 [==============================] - 659s 7s/step - loss: 0.2838 - accuracy: 0.8825 - val_loss: 0.2940 - val_accuracy: 0.9062
Epoch 2/5
100/100 [==============================] - 541s 5s/step - loss: 0.2493 - accuracy: 0.9031 - val_loss: 0.2237 - val_accuracy: 0.9375
Epoch 3/5
100/100 [==============================] - 459s 5s/step - loss: 0.2358 - accuracy: 0.9013 - val_loss: 0.2522 - val_accuracy: 0.9062
Epoch 4/5
100/100 [==============================] - 457s 5s/step - loss: 0.2183 - accuracy: 0.9019 - val_loss: 0.2304 - val_accuracy: 0.9271
Epoch 5/5
100/100 [==============================] - 461s 5s/step - loss: 0.2028 - accuracy: 0.9194 - val_loss:

In [65]:
loss, accuracy = model_from_vvg16.evaluate_generator(test_data_generator, total_validate//batch_size, workers=12)
print("accuracy = %f, loss = %f " % (accuracy, loss))

accuracy = 0.927083, loss = 0.212637 


Fine tuning улучшает качество модели.

## InceptionV3

In [66]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [67]:
train_data_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "../data/cats_dogs", 
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size
)

Found 1600 validated image filenames belonging to 2 classes.


In [69]:
test_data_generator = test_datagen.flow_from_dataframe(
    validate_df, 
    "../data/cats_dogs", 
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size
)

Found 100 validated image filenames belonging to 2 classes.


In [71]:
inceptionV3 = InceptionV3(weights='imagenet', include_top=True)
inceptionV3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [72]:
new_classification_layer = Dense(2, activation='softmax', name="CatsAndDogsPredictions")
out = new_classification_layer(inceptionV3.layers[-2].output)
model_from_inceptionV3 = Model(inceptionV3.input, out)

In [73]:
model_from_inceptionV3.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [74]:
for l, layer in enumerate(model_from_inceptionV3.layers[:-1]):
    layer.trainable = False

model_from_inceptionV3.layers[-1].trainable = True

In [75]:
model_from_inceptionV3.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [76]:
model_from_inceptionV3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
model_from_inceptionV3.fit_generator(train_data_generator, epochs=epochs, validation_data=test_data_generator,
                                     validation_steps=total_validate//batch_size, steps_per_epoch=total_train//batch_size)

C:\Users\danya\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/5
100/100 [==============================] - 115s 1s/step - loss: 0.1451 - accuracy: 0.9400 - val_loss: 0.0789 - val_accuracy: 0.9583
Epoch 2/5
100/100 [==============================] - 113s 1s/step - loss: 0.0421 - accuracy: 0.9850 - val_loss: 0.0502 - val_accuracy: 0.9792
Epoch 3/5
100/100 [==============================] - 115s 1s/step - loss: 0.0424 - accuracy: 0.9856 - val_loss: 0.0630 - val_accuracy: 0.9688
Epoch 4/5
100/100 [==============================] - 114s 1s/step - loss: 0.0410 - accuracy: 0.9844 - val_loss: 0.0481 - val_accuracy: 0.9792
Epoch 5/5
100/100 [==============================] - 113s 1s/step - loss: 0.0371 - accuracy: 0.9856 - val_loss:

In [78]:
loss, accuracy = model_from_inceptionV3.evaluate_generator(test_data_generator, total_validate//batch_size, workers=12)
print("accuracy = %f, loss = %f " % (accuracy, loss))

C:\Users\danya\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
accuracy = 0.989583, loss = 0.046836 


### Fine tuning

In [79]:
for layer in model_from_inceptionV3.layers:
    if layer.name == 'conv2d_93':
        layer.trainable = True
    else:
        layer.trainable = False

model_from_inceptionV3.layers[-1].trainable = True

In [80]:
model_from_inceptionV3.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [81]:
model_from_inceptionV3.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [83]:
model_from_inceptionV3.fit_generator(train_data_generator, epochs=1, validation_data=test_data_generator,
                                     validation_steps=total_validate//batch_size, steps_per_epoch=total_train//batch_size)

100/100 [==============================] - 105s 1s/step - loss: 0.0235 - accuracy: 0.9925 - val_loss: 0.0462 - val_accuracy: 0.9792


In [84]:
loss, accuracy = model_from_inceptionV3.evaluate_generator(test_data_generator, total_validate//batch_size, workers=12)
print("accuracy = %f, loss = %f " % (accuracy, loss))

accuracy = 0.979167, loss = 0.044554 


## Вывод

Fine-Tuning улучшает качество моделей 
 